In [ ]:
import sys
sys.path.append('..')
from datasets import load_dataset
import os 
from utils import resize_preserve_aspect_ratio, pad_to_fixed_size
workers = os.cpu_count()

dataset_id = "jordandavis/fashion_people_detections"
ds = load_dataset(dataset_id, split='train', num_proc=workers)

In [ ]:
def resize_image_pil(image_pil):
    if image_pil.mode != "RGB":
        image_pil = image_pil.convert("RGB")
    image_pil = resize_preserve_aspect_ratio(image_pil, 1024)
    image_pil = pad_to_fixed_size(image_pil, (1024, 1024))
    return image_pil

In [ ]:
def resize_image(examples):
    # Check if examples['image'] is a list of images or a single image
    if isinstance(examples['image'], list):
        # If it's a list, process each image
        examples['image'] = [resize_image_pil(image) for image in examples['image']]
    else:
        # If it's a single image, process the image directly
        examples['image'] = resize_image_pil(examples['image'])
    return examples

In [ ]:
ds = ds.take(1000)

In [ ]:
ds = ds.map(resize_image, batched=True, batch_size=300, with_indices=True)